# A multi layer epithelium

Consisting of stacked 2D sheets

In [22]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tyssue
from tyssue.core.sheet import Sheet
from tyssue.geometry.sheet_geometry import SheetGeometry
from tyssue.geometry.planar_geometry import PlanarGeometry
from tyssue.dynamics.sheet_vertex_model import SheetModel
from tyssue.dynamics.planar_vertex_model import PlanarModel


from scipy.interpolate import Rbf
from tyssue.core.generation import hexa_grid2d, from_2d_voronoi
from scipy.spatial import Voronoi

In [2]:
base_specs = tyssue.config.geometry.sheet_spec()



In [3]:
specs = base_specs.copy()
specs['face']['layer'] = 0
specs['vert']['layer'] = 0
specs['vert']['depth'] = 0.0
specs['edge']['layer'] = 0
specs['settings']['geometry'] = 'flat'

In [5]:
layer1_centers = hexa_grid2d(24, 24, 1, 1, 0.4)
layer2_centers = hexa_grid2d(16, 16, 2, 2, 1)
layer1_data = from_2d_voronoi(Voronoi(layer1_centers))
layer2_data = from_2d_voronoi(Voronoi(layer2_centers))


In [6]:
from tyssue.utils.utils import spec_updater

In [14]:


class MultiSheet():
    
    
    def __init__(self, name, layer_datasets, specs):
        
        self.coords = ['x', 'y', 'z']
        self.layers = [Sheet('{}_{}'.format(name, i),
                             dsets, specs,
                             coords=self.coords)
                       for i, dsets in enumerate(layer_datasets)]
        for i, layer in enumerate(self):
            for dset in layer.datasets.values():
                dset['layer'] = i
        
        

    def __iter__(self):
        for layer in self.layers:
            yield layer
    
    def __getitem__(self, n):
        return self.layers[n]
    
    def __len__(self):
        return len(self.layers)
    
    @property
    def Nes(self):
        return [layer.Ne for i, layer in self]
    
    @property
    def Nvs(self):
        return [layer.Nv for i, layer in self]

    @property
    def Nfs(self):
        return [layer.Nf for i, layer in self]
    
    def concat_datasets(self):
        datasets = {}
        for key in ['edge', 'face', 'vert']:
            datasets[key] = pd.concat([layer.datasets[key]
                                       for layer in self])
        return datasets
    
    def update_interpolants(self):
        self.interpolants = [Rbf(sheet.vert_df['x'],
                                 sheet.vert_df['y'],
                                 sheet.vert_df['z'])
                             for sheet in self]


In [17]:
msheet = MultiSheet('two',
                    [layer1_data,
                     layer2_data],
                    specs)

In [25]:
tyssue.config.dynamics.quasistatic_sheet_spec_anchors()

{'edge': {'anchor_tension': 0.12, 'line_tension': 0.12},
 'face': {'contractility': 0.04,
  'prefered_area': 24.0,
  'prefered_height': 10.0,
  'prefered_vol': 0.0,
  'vol_elasticity': 1.0},
 'settings': {'grad_norm_factor': 1.0, 'nrj_norm_factor': 1.0},
 'vert': {'radial_tension': 0.0}}

In [19]:


class MultiSheetGeometry():
    
    @classmethod
    def update_all(cls, msheet):
        
        msheet.update_interpolants()
        for sheet in msheet:
            SheetGeometry.update_dcoords(sheet)
            SheetGeometry.update_length(sheet)
            SheetGeometry.update_centroid(sheet)
            SheetGeometry.update_normals(sheet)
            SheetGeometry.update_areas(sheet)
            SheetGeometry.update_perimeters(sheet)
            #SheetGeometry.update_vol(sheet)
        cls.update_heights(msheet)
        
    @staticmethod
    def update_heights(msheet):
        
        for lower, upper in zip(msheet.interpolants[:-1], 
                                msheet[1:]):
            upper.vert_df['height'] = (upper.vert_df['z']
                                       - lower(upper.vert_df['x'],
                                               upper.vert_df['y']))
            
        for lower, upper in zip(msheet[:-1], 
                                msheet.interpolants[1:]):
            lower.vert_df['depth'] = (upper(lower.vert_df['x'],
                                            lower.vert_df['y'])
                                      - upper.vert_df['z'])
            

        

In [20]:
MultiSheetGeometry.update_all(msheet)

In [33]:
from copy import deepcopy

class MultiSheetModel():
    
    @staticmethod
    def dimentionalize(mod_specs):
        """
        Changes the values of the input gamma and lambda parameters
        from the values of the prefered height and area.
        Computes the norm factor.
        """

        dim_mod_specs = deepcopy(mod_specs)

        Kv = dim_mod_specs['face']['area_elasticity']
        A0 = dim_mod_specs['face']['prefered_area']
        gamma = dim_mod_specs['face']['contractility']
        kappa_d = dim_mod_specs['vert']['d_elasticity']
        
        dim_mod_specs['face']['contractility'] = gamma * Kv * A0

        lbda = dim_mod_specs['edge']['line_tension']
        dim_mod_specs['edge']['line_tension'] = lbda * Kv * A0**1.5

        dim_mod_specs['settings']['grad_norm_factor'] = Kv * A0**1.5
        dim_mod_specs['settings']['nrj_norm_factor'] = Kv * A0**2

        dim_mod_specs['vert']['d_elasticity'] = kappa_d * Kv * A0
        
        if 'anchor_tension' in dim_mod_specs['edge']:
            t_a = dim_mod_specs['edge']['anchor_tension']
            dim_mod_specs['edge']['anchor_tension'] = t_a * Kv * A0**1.5

        
            
        return dim_mod_specs

    @classmethod
    def compute_energy(cls, msheet):
        
        E = 0
        for sheet in msheet:
            E += PlanarModel.compute_energy(sheet)
        E += cls.desmosome_energy(msheet)
        
        
    @staticmethod
    def desmosome_energy(msheet):
        
        base_sheet = msheet[0]
        basal = base_sheet.eval(
            '0.5 * d_elasticity * ((z - basal_shift) - prefered_height)**2')
        E_d = base_sheet.sum()
        
        for sheet in msheet[1:]:
            upward = sheet.vert_df.eval(
                '0.5 * d_elasticity * (height - prefered_height)**2')
            E_d += upward.sum()
            
        for sheet in msheet[:-1]:
            downward = sheet.vert_df.eval(
                '0.5 * d_elasticity * (depth - prefered_height)**2')
            E_d += downward.sum()
        
        return E_d
    
    
    @staticmethod
    def desmosome_gradient(msheet):
        
        grads = []
        base_sheet = msheet[0]
        grad0 = base_sheet.vert_df[sheet.coords].copy()
        grad0[['x', 'y']] = 0
        grad0['z'] = base_sheet.vert_df.eval(
            'd_elasticity * ((z - basal_shift) - prefered_height)')
        
        for i, sheet in enumerate(msheet[1:]):
            upward = sheet.vert_df.eval(
                'd_elasticity * (height - prefered_height)**2')

        for sheet in msheet[:-1]:
            downward = sheet.vert_df.eval(
                '- d_elasticity * (depth - prefered_height)**2')

    

In [36]:
mod_specs = {
    'edge': {
        'anchor_tension': 0.12,
        'line_tension': 0.12
        },
     'face': {
        'contractility': 0.04,
        'prefered_area': 1.0,
        'prefered_vol': 0.0,
        'area_elasticity': 1.0
        },
     'settings': {
        'grad_norm_factor': 1.0,
        'nrj_norm_factor': 1.0
        },
     'vert': {
        'radial_tension': 0.0,
        'prefered_height': 1.0,
        'd_elasticity': 1.0}
        }


layers_mod_specs = [
    {'face': {'prefered_area': 1.}},
    {'face': {'prefered_area': 4.}}]

for sheet, spec in zip(msheet, 
                       layers_mod_specs):
    
    spec_updater(mod_specs, spec)
    sheet.update_specs(MultiSheetModel.dimentionalize(mod_specs),
                       reset=True)


In [37]:
MultiSheetModel.compute_energy(msheet)

In [16]:
%matplotlib qt
from tyssue.draw.plt_draw import sheet_view


fig, ax = plt.subplots()
draw_spec = {
    'vert':{
        'visible': False},
    'edge':{
        'head_width': 1e-3}
    }

for layer in msheet:
    
    fig, ax = sheet_view(layer, ax=ax, **draw_spec)

In [32]:
dz = 0.1
for i, layer in enumerate(msheet):
    layer.vert_df['z'] = dz * i
    

In [ ]:
msheet

In [ ]:
class MultiSheetGeom(SheetGeometry):
    
    
    